# 📌 DataBench QA System - SemEval Task 8 (Transformer-Based QA Model)

***This project is designed to process dataset files from the SemEval Task 8 competition, clean them, and answer questions from test_qa.csv using a transformer-based question-answering (QA) model. The entire process is automated, from data preprocessing to generating the final predictions in a ZIP archive.***

# Data Processing

🛠️ **Import Necessary Libraries**

In [1]:
import os
import pandas as pd
import ast  # For safely evaluating array-like strings
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
import re
from difflib import get_close_matches
from transformers import pipeline
import warnings
import torch
from concurrent.futures import ProcessPoolExecutor
import zipfile
import logging
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**Cleaning Data using (Data Cleaning Function)**

In [ ]:
def clean_data(df, folder_name):
    """
    Perform data cleaning based on folder-specific requirements.

    Parameters:
    df (pd.DataFrame): The DataFrame to clean.
    folder_name (str): The name of the folder to apply specific cleaning rules.

    Returns:
    pd.DataFrame: The cleaned DataFrame.
    """
    # General cleaning logic
    df = df.copy()
    
    # Apply folder-specific cleaning logic
    if folder_name == "080_Books":
        columns_fill_zero = ['Copies Left', 'Wished Users', 'Reviews']
        df[columns_fill_zero] = df[columns_fill_zero].fillna(0)
        numerical_columns = ['Ratings']
        df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].median())
        categorical_columns = ['Publication']
        df[categorical_columns] = df[categorical_columns].fillna('Unknown')
        text_columns = ['Book Title', 'Author', 'Category', 'Stock Status', 'Edition', 'Publication']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()
        df['Price (TK)'] = pd.to_numeric(df['Price (TK)'], errors='coerce')

    elif folder_name == "079_Coffee":
        text_columns = ['store_location', 'product_category', 'product_type', 'product_detail', 'Month_1', 'Weekday_1']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()
        df['Revenue'] = df['Revenue'].astype(str).str.replace(',', '.').str.strip()
        numeric_columns = ['transaction_qty', 'unit_price']
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    elif folder_name == "078_Fires":
        text_columns = ['calendar_names_1', 'calendar_names_2', 'calendar_1', 'calendar_2']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()
        numeric_columns = ['area', 'DMC', 'DC', 'temp', 'ISI', 'wind']
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    elif folder_name == "076_NBA":
        text_columns = ['Season_type', 'PLAYER', 'TEAM']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()

    elif folder_name == "075_Mortality":
        text_columns = ['Region', 'Status', 'Sex', 'Cause']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()

    elif folder_name == "074_Lift":
        text_columns = ['Lifter Name', 'Weight Class', 'Lift Type']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()

    elif folder_name == "071_COL":
        text_columns = ['Country']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()

    elif folder_name == "070_OpenFoodFacts":
        # Handle Missing Values
        for col in df.columns:
            if df[col].dtype == "object":
                df[col] = df[col].fillna("unknown")
            else:
                df[col] = df[col].fillna(0)

        # Standardize Text Data
        object_columns = df.select_dtypes(include=["object"]).columns
        for col in object_columns:
            df[col] = df[col].str.lower().str.strip()

        # Handle Array-Like Data
        for col in df.columns:
            if df[col].dtype == "object":
                def process_array(value):
                    try:
                        array = ast.literal_eval(value)
                        if isinstance(array, list):
                            if not array:
                                return ["unknown"]
                            return [str(item).lower().strip() if isinstance(item, str) else item for item in array]
                        else:
                            return value
                    except (ValueError, SyntaxError):
                        return value
                df[col] = df[col].apply(process_array)

        # Ensure Correct Data Types
        for col in df.columns:
            if df[col].dtype == "object":
                try:
                    df[col] = pd.to_numeric(df[col], errors="ignore")
                except ValueError:
                    pass

        # Process Specific Columns
        def process_column(entry):
            if isinstance(entry, list):
                return [item.strip().lower() for item in entry if isinstance(item, str)]
            elif isinstance(entry, str):
                entry = entry.strip('[]')  # Remove the outer brackets
                return [item.strip().lower() for item in entry.split(',') if item.strip()]
            else:
                return entry

        columns_to_process = [
            'categories_en', 'states_en', 'brands', 'labels_en', 'stores',
            'countries_en', 'ingredients_analysis_tags', 'ingredients_tags'
        ]

        for column in columns_to_process:
            if column in df.columns:
                df[column] = df[column].apply(process_column)

    elif folder_name == "069_Taxonomy":
        df.drop(columns=['Unnamed: 7'], inplace=True, errors='ignore')
        df.dropna(subset=['Unique ID', 'Parent'], inplace=True)
        for column in ['Tier 2', 'Tier 3', 'Tier 4']:
            if column in df.columns and pd.api.types.is_categorical_dtype(df[column]):
                df[column] = df[column].cat.add_categories('Unknown')
                df[column] = df[column].fillna('Unknown')

        text_columns = ['Name', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4']
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()
        df.drop_duplicates(inplace=True)

    elif folder_name == "068_WorldBank_Awards":
        # Ensure the 'Procurement Method' column is handled as a categorical column
        if 'Procurement Method' in df.columns:
            if pd.api.types.is_categorical_dtype(df['Procurement Method']):
                 # Add 'Unknown' category if not already present
                 if 'Unknown' not in df['Procurement Method'].cat.categories:
                      df['Procurement Method'] = df['Procurement Method'].cat.add_categories('Unknown')
            df['Procurement Method'] = df['Procurement Method'].fillna('Unknown')

        # Ensure 'Project Global Practice' column is handled as a categorical column
        if 'Project Global Practice' in df.columns:
            if pd.api.types.is_categorical_dtype(df['Project Global Practice']):
                # Add 'Not Specified' category if not already present
                if 'Not Specified' not in df['Project Global Practice'].cat.categories:
                     df['Project Global Practice'] = df['Project Global Practice'].cat.add_categories('Not Specified')
            df['Project Global Practice'] = df['Project Global Practice'].fillna('Not Specified')

        
        if 'Borrower Contract Reference Number' in df.columns:
            df['Borrower Contract Reference Number'] = df['Borrower Contract Reference Number'].cat.add_categories('N/A')
            df['Borrower Contract Reference Number'] = df['Borrower Contract Reference Number'].fillna('N/A')

        df['Supplier ID'] = df['Supplier ID'].fillna(-1)
        df['Contract Description'] = df['Contract Description'].fillna('No Description Provided')
        if 'Supplier Country Code' in df.columns:
            df['Supplier Country Code'] = df['Supplier Country Code'].cat.add_categories('Unknown')
            df['Supplier Country Code'] = df['Supplier Country Code'].fillna('Unknown')

        if 'Supplier Country' in df.columns and pd.api.types.is_categorical_dtype(df['Supplier Country']):
             df['Supplier Country'] = df['Supplier Country'].cat.add_categories('Unknown')
             df['Supplier Country'] = df['Supplier Country'].fillna('Unknown')

        if 'Supplier' in df.columns and pd.api.types.is_categorical_dtype(df['Supplier']):
             df['Supplier'] = df['Supplier'].cat.add_categories('Unknown Supplier')
             df['Supplier'] = df['Supplier'].fillna('Unknown Supplier')

        df.drop(columns=['Borrower Country Code'], inplace=True, errors='ignore')
        text_columns = [
            'Procurement Method', 'Project Global Practice', 'Contract Description',
            'Supplier Country Code', 'Borrower Country', 'Region',
            'Supplier Country', 'Supplier', 'Project Name'
        ]
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()
        df['Region'] = df['Region'].str.replace('_', ' ')
        df.drop_duplicates(inplace=True)

    elif folder_name == "067_TripAdvisor":
        def process_text(text):
            lemmatizer = WordNetLemmatizer()
            #stemmer = PorterStemmer() ####### not utilizing it
            text = text.lower().strip()
            tokens = word_tokenize(text)
            return " ".join([
                lemmatizer.lemmatize(token)  # Only lemmatize the tokens
                #stemmer.stem(lemmatizer.lemmatize(token))
                for token in tokens if token not in ENGLISH_STOP_WORDS
            ])
        
        # Ensure 'date_stayed' is processed correctly
        if 'date_stayed' in df.columns:
            if pd.api.types.is_categorical_dtype(df['date_stayed']):
                df['date_stayed'] = df['date_stayed'].cat.add_categories(['Unknown'])
            df['date_stayed'] = df['date_stayed'].fillna('Unknown')
        
        # Process other text columns
        if 'title' in df.columns:
            df['title'] = df['title'].astype(str).fillna("").str.lower().str.strip()

        if 'ratings' in df.columns:
            df['ratings'] = df['ratings'].apply(lambda x: x.lower().strip() if isinstance(x, str) else x)
        if 'text' in df.columns:
            df['text'] = df['text'].apply(process_text)

    elif folder_name == "066_IBM_HR":
        text_columns = df.select_dtypes(include=['object']).columns
        for col in text_columns:
            df[col] = df[col].str.lower().str.strip()



    return df



***Processing Parquet Files***

In [ ]:
def process_parquet_files(base_folder):
    """
    Process parquet files for each folder: read, clean, and save as CSV.

    Parameters:
    base_folder (str): Path to the competition folder.

    Returns:
    None
    """
    for folder_name in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder_name)
        
        if os.path.isdir(folder_path):
            print(f"Processing folder: {folder_name}")
            
            # Process 'all.parquet'
            all_parquet_path = os.path.join(folder_path, "all.parquet")
            all_csv_path = os.path.join(folder_path, "cleaned_all.csv")
            if os.path.exists(all_parquet_path):
                all_df = pd.read_parquet(all_parquet_path)
                cleaned_all_df = clean_data(all_df, folder_name)
                cleaned_all_df.to_csv(all_csv_path, index=False)
                print(f"Processed and saved: {all_csv_path}")
            
            # Process 'sample.parquet'
            sample_parquet_path = os.path.join(folder_path, "sample.parquet")
            sample_csv_path = os.path.join(folder_path, "cleaned_sample.csv")
            if os.path.exists(sample_parquet_path):
                sample_df = pd.read_parquet(sample_parquet_path)
                cleaned_sample_df = clean_data(sample_df, folder_name)
                cleaned_sample_df.to_csv(sample_csv_path, index=False)
                print(f"Processed and saved: {sample_csv_path}")

# Usage
base_folder = r"C:\Users\ASUS\Downloads\competition\competition" # Replace with your actual path
process_parquet_files(base_folder)


# Advanced Question Answering System

***Load Transformer-Based QA Model***

In [6]:
# ========== CONFIGURATION ==========
warnings.filterwarnings("ignore", category=FutureWarning)
DEVICE = 0 if torch.cuda.is_available() else -1
MODEL_NAME = "deepset/bert-large-uncased-whole-word-masking-squad2"  # Larger model
DATA_DIR = "/kaggle/input/competition-csv/competition_csv/competition"
QA_FILE = os.path.join(DATA_DIR, "test_qa.csv")

# Set up advanced logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

print("✅ Loading enhanced QA pipeline...")
qa_pipeline = pipeline(
    "question-answering", 
    model=MODEL_NAME,
    tokenizer=MODEL_NAME,
    device=DEVICE,
    max_seq_len=512,
    doc_stride=128
)

✅ Loading enhanced QA pipeline...


Some weights of the model checkpoint at deepset/bert-large-uncased-whole-word-masking-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


***Semantic Column Matching***
Finding the Best Column for Each Question

In [7]:
# ========== ENHANCED TEXT PROCESSING ==========
class SemanticColumnMatcher:
    def __init__(self, columns):
        self.columns = columns
        self.vectorizer = TfidfVectorizer()
        self.column_vectors = self.vectorizer.fit_transform(
            [self._preprocess(col) for col in columns]
        )
    
    def _preprocess(self, text):
        return re.sub(r'[^\w\s]', '', str(text).lower())
    
    def best_match(self, question):
        question_vec = self.vectorizer.transform([self._preprocess(question)])
        similarities = cosine_similarity(question_vec, self.column_vectors)
        best_idx = np.argmax(similarities)
        return self.columns[best_idx] if similarities[0, best_idx] > 0.3 else None

***Handling Different Question Types***
with Using BERT for QA (feeds the question and context into the BERT model.)

In [8]:

# ========== ADVANCED QUESTION HANDLER ==========
class EnhancedQuestionHandler:
    def __init__(self, df):
        self.df = df.convert_dtypes().infer_objects()
        self.column_matcher = SemanticColumnMatcher(df.columns.tolist())
        self._preprocess_data()
        
    def _preprocess_data(self):
        # Convert potential date columns
        date_cols = [col for col in self.df.columns 
                    if re.search(r'date|year|month|day', col, re.I)]
        for col in date_cols:
            self.df[col] = pd.to_datetime(self.df[col], errors='ignore')
            
        # Enhanced numeric detection
        self.numeric_cols = self.df.select_dtypes(include=np.number).columns.tolist()
        
    def handle(self, question):
        try:
            if self._is_boolean(question):
                return self._handle_boolean(question)
            if self._is_list(question):
                return self._handle_list(question)
            if self._is_aggregate(question):
                return self._handle_aggregate(question)
            if self._is_comparison(question):
                return self._handle_comparison(question)
            return self._handle_fallback(question)
        except Exception as e:
            logging.error(f"Error handling question: {question} - {str(e)}")
            return self._handle_fallback(question)

    def _is_boolean(self, question):
        return re.search(r'\b(is|are|does|do|was|were|has|have|had|can|could|should|will|would)\b', 
                        question, re.I)

    def _is_list(self, question):
        return re.search(r'\b(list|top|most|least|unique|first|last)\b', question, re.I)

    def _is_aggregate(self, question):
        return re.search(r'\b(count|sum|total|average|mean|max|min|median|std|deviation|range)\b', 
                        question, re.I)

    def _is_comparison(self, question):
        return re.search(r'\b(greater|more than|above|less than|below|between|after|before)\b', 
                        question, re.I)

    def _handle_boolean(self, question):
        # Enhanced boolean logic with multi-column support
        col = self.column_matcher.best_match(question)
        if col:
            if re.search(r'\b(all|every)\b', question, re.I):
                return "Yes" if self.df[col].nunique() == 1 else "No"
            if re.search(r'\b(any|exists)\b', question, re.I):
                return "Yes" if not self.df[col].isnull().all() else "No"
            
            numbers = re.findall(r'\d+\.?\d*', question)
            if numbers and col in self.numeric_cols:
                value = float(numbers[0])
                if re.search(r'\b(greater|more than|above)\b', question):
                    return "Yes" if (self.df[col] > value).any() else "No"
                if re.search(r'\b(less than|below)\b', question):
                    return "Yes" if (self.df[col] < value).any() else "No"
                
        return "No"  # Conservative default

    def _handle_list(self, question):
        col = self.column_matcher.best_match(question)
        if not col:
            return None
            
        # Handle top N queries
        n = min([int(num) for num in re.findall(r'\d+', question)] or [5])
        
        if re.search(r'\b(top|highest|most)\b', question):
            if col in self.numeric_cols:
                return self.df.nlargest(n, col)[col].tolist()
            return self.df[col].value_counts().head(n).index.tolist()
            
        if re.search(r'\b(bottom|lowest|least)\b', question):
            if col in self.numeric_cols:
                return self.df.nsmallest(n, col)[col].tolist()
            return self.df[col].value_counts().tail(n).index.tolist()
            
        return self.df[col].dropna().unique().tolist()[:n]

    def _handle_aggregate(self, question):
        col = self.column_matcher.best_match(question)
        if not col or col not in self.numeric_cols:
            return None
            
        if re.search(r'\b(count|number)\b', question):
            return int(self.df[col].count())
            
        if re.search(r'\b(sum|total)\b', question):
            return f"{self.df[col].sum():.2f}"
            
        if re.search(r'\b(average|mean)\b', question):
            return f"{self.df[col].mean():.2f}"
            
        if re.search(r'\b(median)\b', question):
            return f"{self.df[col].median():.2f}"
            
        if re.search(r'\b(max|highest)\b', question):
            return f"{self.df[col].max():.2f}"
            
        if re.search(r'\b(min|lowest)\b', question):
            return f"{self.df[col].min():.2f}"
            
        if re.search(r'\b(range)\b', question):
            return f"{self.df[col].max() - self.df[col].min():.2f}"
            
        return None

    def _handle_comparison(self, question):
        # Handle complex comparisons
        cols = [self.column_matcher.best_match(q_part) 
               for q_part in re.split(r'\b(and|or)\b', question)]
        cols = [c for c in cols if c]
        
        if len(cols) >= 2 and cols[0] in self.numeric_cols:
            numbers = [float(n) for n in re.findall(r'\d+\.?\d*', question)]
            if len(numbers) >= 2:
                return str(self.df[
                    (self.df[cols[0]] > numbers[0]) & 
                    (self.df[cols[1]] < numbers[1])
                ].shape[0])
        
        return None

    def _handle_fallback(self, question):
        # Enhanced context generation for QA model
        context = "\n".join([
            f"Column '{col}': {self._describe_column(col)}"
            for col in self.df.columns
        ])
        
        try:
            result = qa_pipeline(question=question, context=context)
            return self._postprocess_answer(result['answer'])
        except:
            return "Unknown"

    def _describe_column(self, col):
        # Generate rich column descriptions
        dtype = str(self.df[col].dtype)
        sample = self.df[col].dropna().sample(min(3, len(self.df))).tolist()
        
        if pd.api.types.is_numeric_dtype(self.df[col]):
            stats = f"min: {self.df[col].min():.2f}, max: {self.df[col].max():.2f}, mean: {self.df[col].mean():.2f}"
        else:
            stats = f"{self.df[col].nunique()} unique values"
            
        return f"{dtype} column. Sample values: {sample}. Statistics: {stats}"

    def _postprocess_answer(self, answer):
        # Clean up model outputs
        answer = re.sub(r'\s+', ' ', answer).strip()
        numbers = re.findall(r'\d+\.?\d*', answer)
        return numbers[0] if numbers else answer



***Processing the Questions and Saving the Answers***
Finding the answer using the BERT model + column matching.

In [9]:

def process_qa_file(output_filename, file_type, max_rows=None):
    qa_df = pd.read_csv(QA_FILE)
    if max_rows: qa_df = qa_df.head(max_rows)
    
    predictions = []
    
    for _, row in qa_df.iterrows():
        dataset_name = row['dataset']
        question = row['question']
        
        dataset_path = os.path.join(DATA_DIR, dataset_name, f"{file_type}.csv")
        if not os.path.exists(dataset_path):
            predictions.append("Dataset not found")
            continue
            
        try:
            df = pd.read_csv(dataset_path)
            handler = EnhancedQuestionHandler(df)
            answer = handler.handle(question)
            predictions.append(str(answer))
        except Exception as e:
            logging.error(f"Error processing {dataset_name}: {str(e)}")
            predictions.append("Error")
    
    # Save with error handling
    try:
        with open(f"{output_filename}.txt", "w") as f:
            f.write("\n".join(predictions))
    except Exception as e:
        logging.error(f"Failed to save predictions: {str(e)}")

if __name__ == "__main__":
    process_qa_file("predictions", "cleaned_all")
    process_qa_file("predictions_lite", "cleaned_sample", max_rows=20)
    
    # Validate file sizes before zipping
    if os.path.exists("predictions.txt") and os.path.exists("predictions_lite.txt"):
        with zipfile.ZipFile("CUET.zip", "w") as zipf:
            zipf.write("predictions.txt")
            zipf.write("predictions_lite.txt")
        print("✅ Archive created successfully")
    else:
        logging.error("Failed to create archive - prediction files missing")

<ipython-input-8-b613ddb1e61e>:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.df[col] = pd.to_datetime(self.df[col], errors='ignore')
<ipython-input-8-b613ddb1e61e>:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.df[col] = pd.to_datetime(self.df[col], errors='ignore')
<ipython-input-8-b613ddb1e61e>:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.df[col] = pd.to_datetime(self.df[col], errors='ignore')
<ipython-input-8-b613ddb1e61e>:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure

✅ Archive created successfully
